<a href="https://colab.research.google.com/github/Morningspread/Angry-Tweeter/blob/main/Fear_hugging_transforrmer_train_G2000_Dec10%2C2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Positive control - How much is our LSTM architecture is holding us back... 

##Dec 12,2022 we are seeing like a difference of 84% versus 70%... on our fear arousal data training strategy...Questions: 

1.   What are we "supposed" to be seeing with that? 
2.   How would we know? 
3.   Does the Tensorflow truncate happen on both sides? 
4.   Is our labelling right or not....?





# New Section

In [1]:
#It turns out that you need both of these dependencies... 

!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.2 MB/s eta 0:00:00


In [2]:
# To mount the drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
#Installing dependencies... 

import pandas as pd
import numpy as np





#Importing the dataset 

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Sean/Emoclass_Dec2022/Fear_arousal/Emotion datasets/fear_arousal_first2000books_w400.csv")

In [4]:
#Let's look at the value counts here...

df.head()

,Unnamed: 0,text,emotion1,emotion2
0,0,of a safe next instant the street was empty ba...,panic,dread
1,1,aw now horn and pelt our peoples melt in cover...,dread,dread
2,2,an unwatched flame brightened for a moment an...,dread,dread
3,3,sign turn thee from thy morning mist or twili...,dread,dread
4,4,nted hathi oh but my tusks were red at bhurtpo...,horror,dread


In [5]:
#looking at the columns

df.columns

Index(['Unnamed: 0', 'text', 'emotion1', 'emotion2'], dtype='object')

In [6]:
#It would be good to see the full sample size here...

df.shape

(151724, 4)

In [7]:
#Let's see what categories we have here...we only have 5 categories... 

df['emotion2'].value_counts()

fear       71839
concern    20277
dread      18452
anxiety    18202
calm       15875
worry       7079
Name: emotion2, dtype: int64

In [8]:
#Let's see what emotion words are in emotion 1

df['emotion1'].value_counts()

fear           35438
afraid         19448
anxious        10561
calm            9192
concerning      8439
anxiety         7629
frightened      7498
horror          7451
concerned       7416
dread           6722
fearful         5134
concern         4422
peaceful        4043
worry           3604
fright          3286
worried         2691
serene          2640
panic           2582
horrified       1697
frightening     1035
worrying         784
angst             12
Name: emotion1, dtype: int64

In [9]:
#We should update this piece of code...it's another one of those manual liabilities...
#We anticipate that we should have no zero values... 

df.rename(columns = {'emotion2' : 'label_name'}, inplace = True) # renaming the column
def assignNewLabels(label):
    if label == "calm":
      return 0
    elif label == "worry":
      return 1
    elif label == "concern":
      return 2
    elif label == "anxiety":
      return 3
    elif label == "fear":
      return 4
    elif label == "dread":
      return 5
    else:
      return 6
df["label"] = df["label_name"].apply(assignNewLabels)


In [10]:
df['label'].value_counts(ascending = True)

1     7079
0    15875
3    18202
5    18452
2    20277
4    71839
Name: label, dtype: int64

#Remove key emotion words 

In [11]:
#Making a list of all of the emotion responses in a dataset...

words=df['emotion1'].unique().tolist()

#And then remove all the emotion words from the dataset 

for j in words: 
  df['text'] = df['text'].str.replace(j,'emotions')

In [12]:
#dropping the unwanted columns

df.drop(['Unnamed: 0','emotion1','label_name'],axis=1, inplace= True ) 

In [13]:
#I think a cell here to explore what the dataset looks like is going to be important... 
#This looks pretty good... 


df['label'].value_counts()

4    71839
2    20277
5    18452
3    18202
0    15875
1     7079
Name: label, dtype: int64

In [14]:
#This is a training set -> so what do we want in the set? 

df.head()

,text,label
0,of a safe next instant the street was empty ba...,5
1,aw now horn and pelt our peoples melt in cover...,5
2,an unwatched flame brightened for a moment an...,5
3,sign turn thee from thy morning mist or twili...,5
4,nted hathi oh but my tusks were red at bhurtpo...,5


#Cutting out one word on each end... 

In [15]:
#Cutting a single word off of both ends - reassign back to variable

def truncate_words(text, n=1):
    words = text.split()
    return ' '.join(words[n:-n])

df['text'] = df['text'].apply(lambda x: truncate_words(x))

In [16]:
#Splitting the dataset....this looks pretty good...

import numpy as np
import pandas as pd

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [17]:
train, validate, test = train_validate_test_split(df)

In [18]:
train.shape

(91034, 2)

In [19]:
validate.shape

(30344, 2)

In [20]:
test.shape

(30346, 2)

In [21]:
#Converting the pandas dataframe into a HuggingFace dataset... 
import datasets
dataset_train = datasets.Dataset.from_pandas(train)
dataset_validate = datasets.Dataset.from_pandas(validate)
dataset_test = datasets.Dataset.from_pandas(test)

In [22]:
dataset_train

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 91034
})

In [23]:
#Create Dataset Dictionary
data_dict = datasets.DatasetDict({"train":dataset_train, 
                                  "validation":dataset_validate, 
                                  "test": dataset_test})

In [24]:
data_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 91034
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 30344
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 30346
    })
})

In [25]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [27]:
#df['emotions_encoded'] = df['text'].apply(tokenize)
emotions_encoded = data_dict.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
#This is an important piece of code to make sure that we have the right number of labels... 
#On the July 21,2022 run: 6 labels 

from transformers import AutoModelForSequenceClassification
num_labels = 6
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [29]:
emotions_encoded["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [30]:
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
emotions_encoded["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [31]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [32]:
#Hyperparameters to consider here are: number of training epochs
#I don't think that we need to have 15 training epochs, but we will let this thing run for now and see how it goes... 

from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  #load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="no",
                                  disable_tqdm=False)

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"])
trainer.train();

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 91034
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 7115
  Number of trainable parameters = 109486854


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.577600,0.454410,0.835948,0.830111
2,0.383100,0.399443,0.857567,0.855554
3,0.262700,0.403942,0.863960,0.863018


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30344
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30344
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you 

In [ ]:
results = trainer.evaluate()
results

In [ ]:
preds_output = trainer.predict(emotions_encoded["validation"])
preds_output.metrics